# Establish base page

In [ ]:
# Build a pipeline

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os
from selenium import webdriver
from selenium.webdriver.edge.service import Service

# provide path to the Edge driver
s=Service("C:\\Users\sandr\OneDrive\Desktop\Drivers\msedgedriver.exe")
driver = webdriver.Edge(service=s)

# Get the url
url_base = 'https://www.nba.com/stats/draft/combine-anthro/?SeasonYear='

driver.get(url_base)

# Get the page source or source code of the webpage
driver.page_source[:100]



In [ ]:
# Use BeautifulSoup and lxml extension to parse, save in variable nba_soup
nba_soup = BeautifulSoup(driver.page_source, 'lxml')

# Locate the table on the page
table = nba_soup.find('table')

# Obtain the header for each column on the table
# Clean up headers by removing unnecessary characters
headers = []
for i in table.find_all('th'):
    title = i.text.replace('\n', '').replace('\xa0', ' ').strip()
    headers.append(title)
    
# Create a dataframe
nba_stat = pd.DataFrame(columns = headers) 

# Create a for loop to fill nba_stat
for i in table.find_all('tr')[1:]:
    row_data = i.find_all('td')
    row =[j.text for j in row_data]
    length = len(nba_stat)
    nba_stat.loc[length] = row

In [ ]:
nba_stat

# Build a pipeline

In [ ]:
# Make a url list
# Remember: url_base = 'https://www.nba.com/stats/draft/combine-anthro/?SeasonYear='

url_list = []

def find_next_url(url_base):
    a = 2021
    while a >= 2000:
        b = a+1
        new_url = url_base + str(a) + "-" + str(b)[-2:]
        a -=1
        return url_list.append(new_url)
        
# # test
# url = 'https://www.nba.com/stats/draft/combine-anthro/?SeasonYear='

# find_next_url(url)

In [ ]:
for url in url_list:
    # Get the url
    driver.get(url)
    
    # Get the page source or source code of the webpage
    driver.page_source[:100]
    
    # Use BeautifulSoup and lxml extension to parse, save in variable nba_soup
    nba_soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # Locate the table on the page
    table = nba_soup.find('table')
    
    # Obtain the header for each column on the table
    # Clean up headers by removing unnecessary characters
    headers = []
    for i in table.find_all('th'):
        title = i.text.replace('\n', '').replace('\xa0', ' ').strip()
        headers.append(title)
    
    # Create a dataframe
    nba_stat1 = pd.DataFrame(columns = headers)
    
    # Create a for loop to fill nba_stat1
    for i in table.find_all('tr')[1:]:
        row_data = i.find_all('td')
        row =[j.text for j in row_data]
        length = len(nba_stat1)
        nba_stat1.loc[length] = row
    
    # Union of table to base table
    nba_union = pd.concat([nba_stat, nba_stat1], ignore_index=True)

# Clean up data

In [ ]:
# Rename some column names to include units of measure
nba_union = nba_union.rename(columns = {"HEIGHT W/O SHOES":"HEIGHT W/O SHOES (inches)",
                                        "HEIGHT W/ SHOES": "HEIGHT W/ SHOES (inches)",
                                       "STANDING REACH": "STANDING REACH (inches)",
                                        "WEIGHT (LBS)": "WEIGHT (lbs)",
                                       "WINGSPAN": "WINGSPAN (inches)"})
    
# Remove column POS, not significant
nba_union = nba_union.drop(['POS'], axis=1)

# Remove '%' from each row in "BODY FAT %" column
nba_union['BODY FAT %'] = [item.replace('%','') for item in nba_union['BODY FAT %']]


In [ ]:
# Find null counts
null_counts = nba_union.isna().sum()
null_counts

In [ ]:
# No null counts, but there are clearly rows with no value inputs
# Remove these rows in the dataframe

# Identify where '-"
nba_union == '-'

# Change all dash strings values into NA values
nba_union = nba_union.replace(r'-', np.nan, regex=True)

In [ ]:
# Identify where "" 
nba_stat1 == ""

# Change all blank strings values into NA values
nba_union = nba_union.replace(r"", np.nan, regex=True)

In [ ]:
# Check null status again, this time it should show counts of null
null_counts = nba_union.isnull().sum()
null_counts

In [ ]:
# Identify rows that are null
df_null = nba_union[nba_union.isnull().any(axis=1)]
df_null

In [ ]:
# Check dimensions of nba_stat1 before dropping null values
nba_union.shape

In [ ]:
# Drop all null values
nba_union = nba_union.dropna()

# Check dimensions after dropping null values
nba_union.shape

In [ ]:
# Make a function to standardize height and wingspan into inches
def convert_to_inches(measure):
        feet_to_inches = float(measure.split("'")[0])*12
        inches = float(measure.split("'")[1].replace("'","").replace('"', ''))
        total_inches = feet_to_inches + inches
        return total_inches


In [ ]:
# Apply conversion function to each column that needs conversion
nba_union['HEIGHT W/O SHOES (inches)'] = nba_union['HEIGHT W/O SHOES (inches)'].apply(convert_to_inches)
nba_union['HEIGHT W/ SHOES (inches)'] = nba_union['HEIGHT W/ SHOES (inches)'].apply(convert_to_inches)
nba_union['STANDING REACH (inches)'] = nba_union['STANDING REACH (inches)'].apply(convert_to_inches)
nba_union['WINGSPAN (inches)'] = nba_union['WINGSPAN (inches)'].apply(convert_to_inches)

nba_union.head()

In [ ]:
# Now change all "number" columns' dtypes to numeric so that data is more usable
# Check dtype status first
nba_union.info()

In [ ]:
nba_union[["BODY FAT %", "HAND LENGTH (inches)", "HAND WIDTH (inches)",
           "WEIGHT (lbs)"]] = nba_union[["BODY FAT %", "HAND LENGTH (inches)", "HAND WIDTH (inches)",
           "WEIGHT (lbs)"]].apply(pd.to_numeric)

# Check dtype status again to verify
nba_union.info()

## Now data is cleaned and ready for visualization

## Save nba_union dataframe into csv file

In [ ]:
nba_csv = nba_union.to_csv('nba_stats1.csv', index=False)

In [ ]:
# Check that it's saved
df = pd.read_csv('nba_stats1.csv')
df